# Synth 2022 Data

### On OSPC

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt

In [ ]:
data_path = Path('/','home','dicarlo_d','Documents','Code','TurboSuperResultion','.cache', 'Synth2D_0001.hdf5')
data_path.exists()

In [ ]:
from kolmopy.datasets.synth2D import Synth2D

In [ ]:
dset = Synth2D(data_path)

In [ ]:
dset.cite()

In [ ]:
dset.validate()

## Multiple images

In [ ]:
from tqdm import tqdm
from IPython.display import HTML
from matplotlib import animation

In [ ]:
t = None
dset.load_data(t, only_vel_field=False)

In [ ]:
u = dset.u
v = dset.v
w = dset.v
uv = dset.uv
print(uv.shape)
print(dset.t)
print(len(dset.t))
diff = dset.diffusion
print(diff)

In [ ]:
plt.imshow(u[0,:,:])
plt.colorbar()

In [ ]:
fig, axarr = plt.subplots(1,1, figsize=(10,10))

ims = []
for i in tqdm(range(0, dset.uv.shape[0], 8)):
    
    data = dset.uv[i,:,:,0]
    im1 = axarr.imshow(data, animated=True)
    
    ims.append([im1])
    
ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=1000)
plt.close()

# Show the animation
HTML(ani.to_html5_video())

In [ ]:
X, y = td[:]
print(X.shape)
print(y.shape)

In [ ]:
R = td.res

uv = y.view(R, R, 2)     # (R*R, 2)
uv = uv.permute(2, 0, 1) # (R, R, 2) -> (2, R, R)

E, k = phy.energy_spectrum(uv)

# wavenumber
wavenumber = (2*np.pi)*(k*(1/256))

plt.loglog(k_bin, Ekt, color='black', lw=1.5, label=r'$3\mathrm{D}\,\, \mathrm{total}$')
c = 0.015
plt.loglog([min(k[1:]), max(k)], 
           [c*min(k[1:])**(-5./3.), c*max(k)**(-5./3.)],
           color='black', lw=1, ls='dotted')
c = 0.02
plt.loglog([min(k[1:]), max(k)],
           [c*min(k[1:])**(-3.), c*max(k)**(-3.)],
           color='C0', lw=1, ls='dotted')

plt.legend()
plt.show()

In [ ]:
Re = 3000
nu = 1/Re

print('TKE', phy.tkenergy(uv).item())
print('Dissipation', phy.dissipation(uv, viscosity=nu).item())
print('RMSE', phy.rmsvelocity(uv).item())
print('Taylor Micro scale', phy.tmscale(uv, viscosity=nu).item())
print('Taylor-scale Reynolds', phy.tsreynolds(uv, viscosity=nu).item())
print('Kolmogorov Time', phy.ktimescale(uv, viscosity=nu))
print('Kolmogorov Lenght', phy.klenscale(uv, viscosity=nu))
print('Integral scale', phy.intscale(uv))

## PROCESS FILES
### Extract TEXT (X,Y,U,V)

In [ ]:
# img_dir = '../../../Datasets/2021-Turb2D/shifttruth1000-txt.txt/'
# imgs = sorted(glob.glob(img_dir + '*.txt'))

# nimgs = len(imgs)

# Turb2D = np.zeros((nimgs,256,256,4))

# for i, img_path in enumerate(tqdm(imgs)):
#     img = np.loadtxt(open(img_path, "rb"))
#     for v in range(4):
#         Turb2D[i,:,:,v] = img[:,v].reshape(256,256)

# outfile = '../data/2021-Turb2D_velocities.npy'
# np.save(outfile, Turb2D)

### Extract Particles

In [ ]:
# img_dir = '../../../Datasets/2021-Turb2D/particle/'
# imgs = sorted(glob.glob(img_dir + '*.tif'))

# nimgs = len(imgs)

# Turbo2DParticles = np.zeros((nimgs, 256,256))

# for i, img_path in enumerate(tqdm(imgs)):
#     img = plt.imread(img_path)
#     Turbo2DParticles[i,:,:] = img

# outfile = '../data/2021-Turb2D_particles.npy'
# np.save(outfile, Turbo2DParticles)

## Extract Scalar

In [ ]:
# img_dir = '../../../Datasets/2021-Turb2D/scalar/'
# imgs = sorted(glob.glob(img_dir + '*.tif'))

# nimgs = len(imgs)

# Turbo2DScalar = np.zeros((nimgs, 256,256))

# for i, img_path in enumerate(tqdm(imgs)):
#     img = plt.imread(img_path)
#     Turbo2DScalar[i,:,:] = img

# outfile = '../data/2021-Turb2D_scalar.npy'
# np.save(outfile, Turbo2DScalar)

## Visualize Data

In [ ]:
Velocities = np.load('../data/2021-Turb2D_velocities.npy')
Scalars = np.load('../data/2021-Turb2D_scalar.npy')
Particles = np.load('../data/2021-Turb2D_particles.npy')

In [ ]:
img = 10

plt.figure(figsize=(5,5))
plt.subplot(221)
plt.imshow(Velocities[img,:,:,0])
plt.subplot(222)
plt.imshow(Velocities[img,:,:,1])
plt.subplot(223)
plt.imshow(Velocities[img,:,:,2])
plt.subplot(224)
plt.imshow(Velocities[img,:,:,3])

plt.figure(figsize=(5,5))
plt.imshow(Scalars[img,:,:])

plt.figure(figsize=(5,5))
plt.imshow(Particles[img,:,:])

In [ ]:
fig, axarr = plt.subplots(1,2, figsize=(10,10))

ims = []
for i in tqdm(range(100)):
       
    im1 = axarr[0].imshow(Scalars[i,:,:], animated=True)
    im2 = axarr[1].imshow(Particles[i,:,:], animated=True)
    
    ims.append([im1, im2])
    
ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=1000)

plt.close()

# Show the animation
HTML(ani.to_html5_video())

## Process Data

In [ ]:
from turboflow.utils import physics as ph

In [ ]:
n = 10

U = Velocities[n,:,:,2]
V = Velocities[n,:,:,3]

# attention might be V <-> U

vorticity = ph.compute_vorticity(U, V)

plt.figure(figsize=(5,5))
plt.imshow(vorticity.T)
plt.show()

fig, ax = plt.subplots(figsize=(5,5))
ph.plot_field(V, U, step=5, scale=1, ax=ax)
plt.show()

fig, ax = plt.subplots(figsize=(5,5))
ph.plot_field(V, U, step=5, scale=1, img=vorticity, ax=ax)
plt.show()

In [ ]:
fig, axarr = plt.subplots(figsize=(10,10))

ims = []
for i in tqdm(range(100)):
    
    U = Velocities[i,:,:,2]
    V = Velocities[i,:,:,3]

#     vorticity = ph.compute_vorticity(U, V)
       
    im1 = ph.plot_field(V, U, step=5, scale=1, ax=axarr)
    
    ims.append([im1])
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

plt.close()

# Show the animation
HTML(ani.to_html5_video())